### Cells below is for preview the dataset

In [4]:
#@title Data Exploration
data_name = "TJ v1.5 - v1.0.2" #@param ["14lap", "14res", "15res", "16res"]

import sys
sys.path.append("aste")
from data_utils import Data

path = f"aste/data/triplet_data/{data_name}/train.txt"
data = Data.load_from_full_path(path)

for s in data.sentences[:3]:
    print("tokens:", s.tokens)
    for t in s.triples:
        print("target:", (t.t_start, t.t_end))
        print("opinion:", (t.o_start, t.o_end))
        print("label:", t.label)
    print()

tokens: ['Ada', 'gak', 'sih', 'yang', 'punya', 'keinginan', 'aneh', 'kayak', 'saya', '?', 'Pengen', 'kerja', 'di', 'pt_transjakarta', 'sejak', 'pelayanannya', 'sebagus', 'itu', '.', 'Padahal', 'saya', 'udah', 'punya', 'pekerjaan', 'tetap', '.']
target: (15, 15)
opinion: (16, 17)
label: LabelEnum.positive

tokens: ['pt_transjakarta', 'TERIMA', 'KASIH', 'semoga', 'bis2', 'yg', 'lain', 'juga', 'secara', 'berkala', 'bisa', 'ttp', 'dibersihkan', 'utk', 'mencegah', 'keluhan2', 'lagi', 'kedepannya']
target: (0, 0)
opinion: (1, 2)
label: LabelEnum.positive
target: (4, 4)
opinion: (10, 12)
label: LabelEnum.positive

tokens: ['Untung', 'busway', 'ac', 'nya', 'adem']
target: (2, 2)
opinion: (4, 4)
label: LabelEnum.positive



In [3]:
import requests
import tarfile
from pathlib import Path

# Define URL and file names
template = "https://github.com/chiayewken/Span-ASTE/releases/download/v1.0.0/{}.tar"
data_name = "14lap"  # Replace with the correct name
url = template.format(data_name)
model_tar = Path(url).name
model_dir = Path(url).stem

# # Step 1: Download the file
# print("Downloading the file...")
# with open(model_tar, "wb") as f:
#     response = requests.get(url, stream=True)
#     if response.status_code == 200:
#         for chunk in response.iter_content(chunk_size=8192):
#             f.write(chunk)
# print("Download complete.")

# # Step 2: Extract the file
# print("Extracting the file...")
# with tarfile.open(model_tar, "r") as tar:
#     tar.extractall(model_dir)
# print(f"Extraction complete. Files are in '{model_dir}' folder.")

In [26]:
print(model_tar)
print(model_dir)
print(path)
print(data)

model.tar
tj2-seed_4
aste/data/triplet_data/tj_2/train.txt
root=WindowsPath('aste/data/triplet_data/tj_2') data_split=<SplitEnum.train: 'train'> sentences=[Sentence(tokens=['Supir', 'bus', 'Transjakarta', 'sangat', 'ramah', 'dalam', 'melayani', 'penumpang', 'berkebutuhan', 'khusus.'], pos=[], weight=1, id=0, is_labeled=True, triples=[SentimentTriple(o_start=3, o_end=4, t_start=0, t_end=1, label=<LabelEnum.positive: 'POS'>)], spans=[]), Sentence(tokens=['Waktu', 'tunggu', 'di', 'halte', 'Harmoni', 'terlalu', 'lama', 'dan', 'tidak', 'ada', 'informasi', 'kedatangan', 'bus.'], pos=[], weight=1, id=0, is_labeled=True, triples=[SentimentTriple(o_start=5, o_end=6, t_start=0, t_end=1, label=<LabelEnum.negative: 'NEG'>), SentimentTriple(o_start=8, o_end=10, t_start=0, t_end=1, label=<LabelEnum.negative: 'NEG'>)], spans=[]), Sentence(tokens=['Bus', 'Transjakarta', 'sangat', 'nyaman', 'dengan', 'kursi', 'empuk', 'dan', 'AC', 'yang', 'dingin.'], pos=[], weight=1, id=0, is_labeled=True, triples=[Se

### for Self-training

In [ ]:
"""
Gunakan SpanModel Anda untuk memberi pre-annotasi,
lalu simpan output sebagai JSON array yang bisa langsung
di-import ke Label Studio (Upload tasks ▸ JSON).
"""

import json, uuid
from pathlib import Path
from wrapper import SpanModel           # ← dari repo ASTE Anda
from data_utils import Data, Sentence, SplitEnum

# --------------------- konfigurasi -----------------------
MODEL_DIR  = "outputs/TJ - Overall Dataset/seed_4 (baseline self_training)"
RAW_TXT    = "self_training/batch 1.txt"            # satu tweet per baris
OUT_FILE   = "self_training/batch 1.json"     # **JSON array!**
LABELS     = {"POS": "sent:POS", "NEU": "sent:NEU", "NEG": "sent:NEG"}
# ---------------------------------------------------------

model = SpanModel(save_dir=MODEL_DIR, random_seed=0)

def predict(text: str):
    """Jalankan SpanModel dan kembalikan (tokens, triples)."""
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False,
                    weight=1, id=0)
    tmp_in, tmp_out = "_tmp_in.txt", "_tmp_out.txt"
    Data(root=Path(), data_split=SplitEnum.test, sentences=[sent]).save_to_path(tmp_in)
    model.predict(tmp_in, tmp_out)
    new_sent = Data.load_from_full_path(tmp_out).sentences[0]
    return new_sent.tokens, new_sent.triples

tasks = []
task_id = 0

with open(RAW_TXT, encoding="utf-8") as f:
    for line in f:
        text = line.strip()
        if not text:
            continue

        tokens, triples = predict(text)
        results = []

        # buat span & relation result
        for t in triples:
            # span Aspect
            asp_id = str(uuid.uuid4())[:10]
            results.append({
                "id": asp_id,
                "from_name": "label",
                "to_name": "tweet_text",
                "type": "labels",
                "origin": "model",
                "value": {
                    "start": len(" ".join(tokens[:t.t_start])),
                    "end": len(" ".join(tokens[:t.t_end+1])),
                    "text": " ".join(tokens[t.t_start:t.t_end+1]),
                    "labels": ["aspect"]
                }
            })

            # span Opinion
            op_id  = str(uuid.uuid4())[:10]
            results.append({
                "id": op_id,
                "from_name": "label",
                "to_name": "tweet_text",
                "type": "labels",
                "origin": "model",
                "value": {
                    "start": len(" ".join(tokens[:t.o_start])),
                    "end": len(" ".join(tokens[:t.o_end+1])),
                    "text": " ".join(tokens[t.o_start:t.o_end+1]),
                    "labels": ["opinion"]
                }
            })

            # relation Aspect ↔ Opinion
            results.append({
                "from_id": asp_id,
                "to_id":   op_id,
                "type":    "relation",
                "direction": "right",
                "labels":  [LABELS[t.label.value]]
            })

        # satu objek = satu task LS
        task_id += 1
        tasks.append({
            "id": task_id,
            "data": {
                "cleaned_full_text": text   # sesuaikan dgn <Text... value="$cleaned_full_text"/>
            },
            "predictions": [{
                "model_version": "spanaste_v1",
                "result": results
            }]
        })

# ----------- simpan JSON array -----------
with open(OUT_FILE, "w", encoding="utf-8") as f:
    json.dump(tasks, f, ensure_ascii=False, indent=2)

print(f"✅  {len(tasks)} tasks tertulis ke {OUT_FILE}")


### Below cells are for testing the model with our own input

In [6]:
model_tar = "model.tar"
model_dir = "outputs/Variasi Dataset/single word"

In [7]:
# Use pretrained SpanModel weights for prediction

import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

def predict_sentence(text: str, model: SpanModel) -> Sentence:
    path_in = "temp_in.txt"
    path_out = "temp_out.txt"
    sent = Sentence(tokens=text.split(), triples=[], pos=[], is_labeled=False, weight=1, id=0)
    data = Data(root=Path(), data_split=SplitEnum.test, sentences=[sent])
    data.save_to_path(path_in)
    model.predict(path_in, path_out)
    data = Data.load_from_full_path(path_out)
    return data.sentences[0]

text = "buset nih busway arah ke blok M lama bener datengnya"

model = SpanModel(save_dir=model_dir, random_seed=0)
sent = predict_sentence(text, model)

for t in sent.triples:
    target = " ".join(sent.tokens[t.t_start:t.t_end+1])
    opinion = " ".join(sent.tokens[t.o_start:t.o_end+1])
    print()
    print(dict(target=target, opinion=opinion, sentiment=t.label))


################################################################################
################################################################################
{'span_model_unused_keys': dict_keys(['serialization_dir'])}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys([])}
{'locals': {'self': ProperRelationExtractor(), 'make_feedforward': <function SpanModel.__init__.<locals>.make_feedforward at 0x0000016D8D2CC4C8>, 'span_emb_dim': 1556, 'feature_size': 20, 'spans_per_word': 0.5, 'positive_label_weight': 1.0, 'regularizer': None, 'use_distance_embeds': True, 'use_pruning': True, 'kwargs': {}, 'vocab': Vocabulary with namespaces:  None__ner_labels, Size: 3 || None__relation_labels, Size: 3 || Non Padded Namespaces: {'*tags', '*labels'}, '__class__': <class 'span_model.models.relation_proper.ProperRelationExtractor'>}}
{'token_emb_dim': 768, 'span_emb_dim': 1556, 'relation_scorer_d

reading instances: 0it [00:00, ?it/s]

ValidationError: 1 validation error for Data
sentences -> 0
  value is not a valid dict (type=type_error.dict)

### Below cell is for training the model

In [1]:
data_name = "TJ - Overall Dataset"

import sys
sys.path.append("aste")
from pathlib import Path
from data_utils import Data, Sentence, SplitEnum
from wrapper import SpanModel

# Train SpanModel from scratch
random_seed = 4
path_train = f"aste/data/triplet_data/{data_name}/train.txt"
path_dev = f"aste/data/triplet_data/{data_name}/dev.txt"
save_dir = f"outputs/{data_name}/seed_{random_seed}"

model = SpanModel(save_dir=save_dir, random_seed=random_seed)
model.fit(path_train, path_dev)

{'weights_dir': WindowsPath('outputs/TJ - Overall Dataset/seed_4/weights')}


error loading _jsonnet (this is expected on Windows), treating training_config/config.jsonnet as plain json
error loading _jsonnet (this is expected on Windows), treating snippet as plain json


2025-05-05 16:54:05,908 - INFO - allennlp.common.params - random_seed = 4
2025-05-05 16:54:05,908 - INFO - allennlp.common.params - numpy_seed = 4
2025-05-05 16:54:05,908 - INFO - allennlp.common.params - pytorch_seed = 4
2025-05-05 16:54:05,983 - INFO - allennlp.common.checks - Pytorch version: 1.7.1+cu101
2025-05-05 16:54:05,983 - INFO - allennlp.common.params - type = default
2025-05-05 16:54:05,983 - INFO - allennlp.common.params - dataset_reader.type = span_model
2025-05-05 16:54:05,983 - INFO - allennlp.common.params - dataset_reader.lazy = False
2025-05-05 16:54:05,991 - INFO - allennlp.common.params - dataset_reader.cache_directory = None
2025-05-05 16:54:05,991 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2025-05-05 16:54:05,991 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2025-05-05 16:54:05,991 - INFO - allennlp.common.params - dataset_reader.manual_multi_process_sharding = False
2025-05-05 16:54:06,002 - INFO 

reading instances: 0it [00:00, ?it/s]

2025-05-05 16:54:14,232 - INFO - allennlp.training.util - Reading validation data from D:\KULIAH\Semester 6\MBKM\Bangkit2024\Entrepeneur Track\Unsmoke\Machine Learning\TA\Span-ASTE\outputs\TJ - Overall Dataset\seed_4\temp_data\dev.json


...

2025-05-05 19:49:50,199 - INFO - allennlp.training.tensorboard_writer -                               Training |  Validation
2025-05-05 19:49:50,201 - INFO - allennlp.training.tensorboard_writer - MEAN__relation_f1         |     0.831  |     0.387
2025-05-05 19:49:50,205 - INFO - allennlp.training.tensorboard_writer - MEAN__relation_precision  |     0.849  |     0.393
2025-05-05 19:49:50,209 - INFO - allennlp.training.tensorboard_writer - MEAN__relation_recall     |     0.814  |     0.382
2025-05-05 19:49:50,211 - INFO - allennlp.training.tensorboard_writer - _MEAN__ner_f1             |     0.948  |     0.617
2025-05-05 19:49:50,214 - INFO - allennlp.training.tensorboard_writer - _MEAN__ner_precision      |     0.955  |     0.617
2025-05-05 19:49:50,218 - INFO - allennlp.training.tensorboard_writer - _MEAN__ner_recall         |     0.941  |     0.617
2025-05-05 19:49:50,220 - INFO - allennlp.training.tensorboard_writer - _None__ner_f1             |     0.948  |     0.617
2025-05-05 19:

  0%|          | 0/1764 [00:00<?, ?it/s]

2025-05-05 19:59:12,771 - INFO - allennlp.training.trainer - Validating


  0%|          | 0/379 [00:00<?, ?it/s]

2025-05-05 19:59:34,168 - INFO - allennlp.training.trainer - Ran out of patience.  Stopping training.
2025-05-05 19:59:34,170 - INFO - allennlp.training.checkpointer - loading best weights
2025-05-05 19:59:34,809 - INFO - allennlp.commands.train - To evaluate on the test set after training, pass the 'evaluate_on_test' flag, or use the 'allennlp evaluate' command.
2025-05-05 19:59:34,811 - INFO - allennlp.common.util - Metrics: {
  "best_epoch": 14,
  "peak_worker_0_memory_MB": 0,
  "peak_gpu_0_memory_MB": 2210.8876953125,
  "training_duration": "2:55:26.102773",
  "training_start_epoch": 0,
  "training_epochs": 16,
  "epoch": 16,
  "training__None__ner_precision": 0.9547685086409217,
  "training__None__ner_recall": 0.9409167367535745,
  "training__None__ner_f1": 0.9477920152493912,
  "training__MEAN__ner_precision": 0.9547685086409217,
  "training__MEAN__ner_recall": 0.9409167367535745,
  "training__MEAN__ner_f1": 0.9477920152493912,
  "training__None__relation_precision": 0.8486707566

### Cell below is for validating the model

In [6]:
# Evaluate SpanModel F1 Score
import json

data_name = "TJ - Overall Dataset"

path_pred = "pred.txt"
path_test = f"aste/data/triplet_data/{data_name}/test.txt"
model.predict(path_in=path_test, path_out=path_pred)
results = model.score(path_pred, path_test)
print(json.dumps(results, indent=2))

################################################################################
################################################################################
{'span_model_unused_keys': dict_keys(['serialization_dir'])}
{'locals': ('span_extractor_type', 'endpoint')}
{'locals': ('use_span_width_embeds', True)}
{'ner_loss_fn': CrossEntropyLoss()}
{'unused_keys': dict_keys([])}
{'locals': {'self': ProperRelationExtractor(), 'make_feedforward': <function SpanModel.__init__.<locals>.make_feedforward at 0x000001A41BF19798>, 'span_emb_dim': 1556, 'feature_size': 20, 'spans_per_word': 0.5, 'positive_label_weight': 1.0, 'regularizer': None, 'use_distance_embeds': True, 'use_pruning': True, 'kwargs': {}, 'vocab': Vocabulary with namespaces:  None__ner_labels, Size: 3 || None__relation_labels, Size: 3 || Non Padded Namespaces: {'*labels', '*tags'}, '__class__': <class 'span_model.models.relation_proper.ProperRelationExtractor'>}}
{'token_emb_dim': 768, 'span_emb_dim': 1556, 'relation_scorer_d

reading instances: 0it [00:00, ?it/s]

{
  "path_pred": "pred.txt",
  "path_gold": "aste/data/triplet_data/TJ - Overall Dataset/test.txt",
  "precision": 0.555992141453831,
  "recall": 0.5062611806797853,
  "score": 0.5299625468164794
}
